In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
%matplotlib inline
import numpy as np
import operator
from matplotlib import pyplot as plt
from repositories.CorpusRepository import *
from repositories.TokenRepository import *
from repositories.EstadisticasRepository import *

path = '/Users/macbookpro/code/RI-Tratamiento-y-Analisis-del-Texto/demo-data/'
corpusRepository = CorpusRepository(path)
tokenRepository = TokenRepository()
pathVacias = None

documentos = corpusRepository.getListDocuments()
response = tokenRepository.tokenizar(documentos,pathVacias = pathVacias)

lista = sorted(response['terminos'].items(), key=operator.itemgetter(1) ,reverse=True)

frecuencias = []
for termino in lista:
    frecuencias.append(termino[1]['CF'])

ranking = xrange(1, len(frecuencias) + 1)

100% ▏██████████████████████████████████████████████████▕



IOError: [Errno 2] No such file or directory: 'results/terminos.txt'

Para cumplir con las consignas definimos una función que tenga los parametros: Ranking, frecuencias y la frecuencia buscasda y devuelva la posición en el ranking que la frecuencias sean menor a la frecuencia buscada.

#### Lógica:
1. Pasamos los tres parametros a logaritmo para hacer la busqueda más sencilla.
2. Buscamos la recta de ajuste para los logaritmos de las frecuencias y ranking para obtener la pendiente y ordenada.
3. Teniendo en mente la función FBL = xP + O donde:
 FBL = Frecuencia Búscada Logarítmica
 P = Pendiente
 O = Ordenada al origen
4. Se cuncluye que x = (FBL - O) / P
5. Volvemos a pasar a representación lineal

In [2]:
def getRankingFrecuencias(ranking, frecuencias, frecuencia_buscada):
    x_log = np.log10(ranking)
    y_log = np.log10(frecuencias)
    recta_ajuste = np.polyfit(x_log, y_log, 1)
    pendiente = recta_ajuste[0]
    ordenada = recta_ajuste[1]
    frecuencia_buscada_log = np.log10(frecuencia_buscada)
    
    # xP + O = FBL -> x = (FBL - O) / P
    ranking_buscado_log = (frecuencia_buscada_log - ordenada) / pendiente
    
    # Lo pasamos a escala lineal
    return int(pow(10, ranking_buscado_log))

frecuencia_buscada = 4
x = getRankingFrecuencias(ranking, frecuencias, frecuencia_buscada)
print "X = " + str(x)

X = 3734


Defino una función que suma las frecuencias de todos los términos con un ranking igual o superior al establecido, para poder determinar qué porcentaje de las frecuencias seria omitido.

Con el valor obtenido en X recorremos las frecuencuas hasta llegar a la posición obtenida y sumamos las frecuencias para luego comparar con lo omitido realmente.

#### Lógica

1. Enumeramos las frecuencias y la recorremos
2. Por cada item, si la posición es mayor al ranking buscado sumamos la frecuencia
3. Devolvemos el porcentaje que representa del total

In [3]:
frecuencia_total = sum(frecuencias)
suma_frecuencia_buscada = 0
for posicion, frecuencia in enumerate(frecuencias):
    if posicion + 1 >= x:
        suma_frecuencia_buscada += frecuencia
porcentajeZipf = (suma_frecuencia_buscada / float(frecuencia_total))* 100

print "PORCENTAJE OMITIDO ZIPF: %.2f%%" % porcentajeZipf

PORCENTAJE OMITIDO ZIPF: 4.88%


Procedemos a calcular la suma de las frecuencias que tienen frecuencia menor a 5

In [4]:
frecuencia_total = sum(frecuencias)
suma_frecuencia_buscada = 0
for frecuencia in frecuencias:
    if frecuencia <= frecuencia_buscada:
        suma_frecuencia_buscada += frecuencia
porcentajeReal = suma_frecuencia_buscada / float(frecuencia_total) * 100

print "PORCENTAJE OMITIDO REAL: %.2f%%" % porcentajeReal 

PORCENTAJE OMITIDO REAL: 5.06%


### Conclusión

Comparando ambos valores ZIPF = 4.88 y REAL = 5.06% se observa que son muy similares. Confirmando la predicción de la ley de Zipf.